<a href="https://colab.research.google.com/github/InstitutodaVinci3/Vancouver/blob/master/Producao_Diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Resumo dos Trabalhos de Coleta, Limpeza e Entendimento dos dados

##Biblotecas Utilizadas

In [21]:
# importando as bibliotecas básicas e preparando ambiente
import pandas as pd                     # convenção 
import numpy as np                      # convenção 
import random as rd                     # rd não é alias padrão mas sim adotado aqui somente 
# Itens adicionais
import glob
import os
import re

##Coleta e Limpeza dos Dados

###Passo 1 - montagem do drive para colocação dos arquivos recebidos

In [22]:
# montagem do Google Drive para acessar os arquivos que foram disponibilizados
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Passo 2 - definição da varíavel "Filenames" para ser utilizado no "for" de leitura, de forma a correr todoos os arquivos do tipo TXT existentes no caminho (path)

In [23]:
# Defifinindo a varíavel FIlenames para ser utilizado no "for" de leitura
path =r'/content/drive/MyDrive/MedMep/Prod_Diaria/'
filenames = glob.glob(path + "/*.txt")
filenames

['/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170508T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170505T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170503T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170702T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170704T173501.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170707T173501.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170709T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170708T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170706T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170701T173500.txt',
 '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170627T173501.txt',
 '/content/drive/MyDr

###Passo 3 - Definição de como capturar a data que está no nome do arquivo
- É necessário capturar a data inserindo-a em uma coluna adicional para que possamos utilizar os dados de produção de leite em conjunto com os dados de temperatura
- Foi necessário tratar da o formato da data capturado

In [24]:
# Ler parte do nome do arquivo
b = re.search(r'(\d+)', '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170901T173500.txt')
print(b[1])

#Formatando Data
import datetime
datetime.datetime.strptime("20170901", "%Y%m%d").strftime("%d/%m/%Y")

20170901


'01/09/2017'

###Passo 4 - Leitura dos arquivos
- Foram disbonibilizados 177 arquivos no formato TXT, Nomenclatura: Produção diária TXT_AAAAMMDDTHHMMSS
- Foi necessário descartar a primeira linha (cabeçalho) de todos os arquivos por falta de padrão nos arquivos enviados
- Foi necessário remover as duas últimas linhas que comtinham comnandos de "end line" e não seriam necessários ao ralizarmos a concatenação dos arquivos
- Foi necessário criar uma nova definição para o cabeçalho do DataFrame
- Foi necessário criar uma nova coluna para a inserção da data da coleta que estava no nome do arquivo
- Os 177 arquivos foram lidos e concatenados gerando um único DataFrame unificado com todos os dados da Produção Diária de leit

In [25]:
# Lendo os dados já com removendo o cabeçalho e rodapés dos arquivos
# Necessário criar o "Cabeçalho" do DataFrame conforme comando asseguir:
Prod_Dia_df = pd.DataFrame({'Grupo':[], 'Id_Animal':[], 'Num_Lac':[], 'Dias_Lac':[], 'Pico':[], 'Hora_S1':[],
                            'Hora_S2':[], 'Hora_S3':[], 'Durac_S1':[], 'Durac_S2':[], 'Durac_S3':[],
                            'Prod_S1':[], 'Prod_S2':[], 'Prod_S3':[], 'Hora_S1_ont':[], 'Hora_S2_ont':[], 
                            'Hora_S3_ont':[], 'Dura_S1_ont':[], 'Dura_S2_ont':[], "Dura_S3_ont":[],
                            'Prod_S1_ont':[], 'Prod_S2_ont':[], 'Prod_S3_ont':[]})

# Início do looping de leitura dos arquivos
for filename in filenames:

# Leitura dos arquivos já com o novo cabeçalho definido:
  a = pd.read_csv(filename, skiprows=1, skipfooter=2, sep='\t', encoding='latin1', engine='python', decimal=',',   
                  names=['Grupo', 'Id_Animal', 'Num_Lac', 'Dias_Lac', 'Pico', 'Hora_S1', 'Hora_S2', 'Hora_S3',
                         'Durac_S1', 'Durac_S2', 'Durac_S3', 'Prod_S1', 'Prod_S2', 'Prod_S3',
                         'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
                         'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'])
# Criação da coluna adicional com a data contida no nome do arquivo:
  a['Data']= datetime.datetime.strptime(re.search(r'(\d+)', filename)[1], "%Y%m%d").strftime("%d/%m/%Y") 

# Concatenando os arquivos lidos dentro do looping 
  Prod_Dia_df = pd.concat([Prod_Dia_df, a])
# Fim do looping de leitura

Prod_Dia_df.head()

,Grupo,Id_Animal,Num_Lac,Dias_Lac,Pico,Hora_S1,Hora_S2,Hora_S3,Durac_S1,Durac_S2,Durac_S3,Prod_S1,Prod_S2,Prod_S3,Hora_S1_ont,Hora_S2_ont,Hora_S3_ont,Dura_S1_ont,Dura_S2_ont,Dura_S3_ont,Prod_S1_ont,Prod_S2_ont,Prod_S3_ont,Data
0,1.0,5402.0,0.0,,,,,,,,,,,,,,,,,,,,,08/05/2017
1,1.0,5408.0,0.0,,,,,,,,,,,,,,,,,,,,,08/05/2017
2,1.0,4584.0,0.0,,,,,,,,,,,,,,,,,,,,,08/05/2017
3,1.0,4585.0,0.0,,,,,,,,,,,,,,,,,,,,,08/05/2017
4,1.0,4588.0,0.0,,,,,,,,,,,,,,,,,,,,,08/05/2017


In [26]:
# Exportando dados dos arquivos unificados mas não limpos para CSV
Prod_Dia_df.to_csv("/content/drive/MyDrive/MedMep/Prod_Diaria/Prod_Diaria_Unif_Sujo.csv")

###Limpeza do arquivo unificado
- Remoção das colunas com dados não pertinentes ou que estavam em branco:
- Informações sobre o Grupo não dizem nada ao nosso estudo (não se sabe o critério utilizado no agrupamento), 
- Numero de dias de lactação. Dado existe somente para poucos animais,
- Pico de produção. Dado existe somente para poucos animais e não se sabe o seu significado,
- Dura_Sx. Refere-se a duração da sessão de ordenha. Dado existe somente para poucas sessões de ordenha,
- Dura_Sx_ont. Refere-se a duração da sessão de ordenha do dia anterior. Dado existe somente para poucas sessões de ordenha,
- Prod_Sx_ont. Refere-se a produção da sessão Sx do dia anterior. Dado existe somente para poucas sessões e animais.

In [27]:
Prod_Dia_limpo_df = Prod_Dia_df.drop(columns=['Grupo', 'Num_Lac', 'Dias_Lac', 'Pico', 'Durac_S1', 'Durac_S2', 'Durac_S3', 
                                              'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
                                              'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'])
# Prod_Dia_limpo_df.head()

# Prod_Dia_df.drop(columns=['Grupo', 'Num_Lac', 'Dias_Lac', 'Pico', 'Durac_S1', 'Durac_S2', 'Durac_S3', 
#          'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
#          'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'], inplace=True)
# Prod_Dia_df.head()

In [28]:
# Exibindo as primeiras linhas do arquivo
Prod_Dia_limpo_df.head()

,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,5402.0,,,,,,,08/05/2017
1,5408.0,,,,,,,08/05/2017
2,4584.0,,,,,,,08/05/2017
3,4585.0,,,,,,,08/05/2017
4,4588.0,,,,,,,08/05/2017


- Hora_Sx refere-se a hora que se iniciou a sessão de ordenha X no formato HH:00
- Prod_Sx refere-se a produção de leite da sessão X em KG

In [29]:
# Exibindo as últimas linhas do arquivo
Prod_Dia_limpo_df.tail()

,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
67,4453.0,,,,,,,25/09/2017
68,3538.0,,,,,,,25/09/2017
69,3460.0,,,,,,,25/09/2017
70,2483.0,,,,,,,25/09/2017
71,2431.0,,,,,,,25/09/2017


###Salvando os dados limpos em arquivo para utilização futura
- Será salvo no formato CSV de maneira a facilitar a leitura nas próximas etapas

In [30]:
# Exportando dados dos arquivos unificados e limpos para CSV
Prod_Dia_df.to_csv("/content/drive/MyDrive/MedMep/Prod_Diaria/Prod_Diaria_Unificado.csv")

##Entendendo os dados


###Estatística descritiva - análise inicial

In [31]:
df=pd.read_csv('https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/Prod_Diaria_Unificado.csv')
df.head()

,Unnamed: 0,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,0,653.0,07:47,15:26,,15.84,7.88,,01/09/2017
1,1,2623.0,08:09,15:43,,13.81,7.66,,01/09/2017
2,2,2624.0,07:59,15:35,,12.63,10.54,,01/09/2017
3,3,2662.0,09:31,,,13.64,,,01/09/2017
4,4,3652.0,07:47,15:26,,12.73,6.80,,01/09/2017


In [41]:
df.shape

(4032, 9)

In [39]:
df.describe()

,Unnamed: 0,Id_Animal
count,4032.000000,4032.000000
mean,35.500000,3969.930556
std,20.785183,926.571380
min,0.000000,653.000000
25%,17.750000,3471.250000
50%,35.500000,4459.000000
75%,53.250000,4557.750000
max,71.000000,5409.000000


- A função Describe não realizou as estastíticas para as colunas Prod_Sx (ñ estão sendo entendidos como numéricos?)

In [32]:
df.Prod_S1.describe()

count                          4032
unique                          972
top                                
freq                           2066
Name: Prod_S1, dtype: object

In [36]:
df.Prod_S2.describe()

count                          4032
unique                          715
top                                
freq                           2170
Name: Prod_S2, dtype: object

In [37]:
df.Prod_S3.describe()

count                          4032
unique                           29
top                                
freq                           4004
Name: Prod_S3, dtype: object

###Dashboard

In [33]:
df=pd.read_csv('https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/Prod_Diaria_Unificado.csv')
df.head()

,Unnamed: 0,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,0,653.0,07:47,15:26,,15.84,7.88,,01/09/2017
1,1,2623.0,08:09,15:43,,13.81,7.66,,01/09/2017
2,2,2624.0,07:59,15:35,,12.63,10.54,,01/09/2017
3,3,2662.0,09:31,,,13.64,,,01/09/2017
4,4,3652.0,07:47,15:26,,12.73,6.80,,01/09/2017


In [34]:
# Producao_S1 = Prod_Dia_df.groupby('Id_Animal').Prod_S1.sum()
# Producao_S1

In [35]:
# Exportando dados dos arquivos unificados para Excel
# Producao_S1.to_excel('/content/drive/MyDrive/MedMep/Prod_Diaria/Producao-S1.xlsx')